Performing node2vec, HOPE and SDNE on datasets(Karate, Dolphin, LFR, American_football, book)

In [1]:
import networkx as nx
from networkx import to_numpy_array
from node2vec import Node2Vec
import numpy as np
from sklearn.cluster import KMeans
from sknetwork.data import karate_club
from sknetwork.clustering import get_modularity
from sknetwork.visualization import svg_graph
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

c:\Users\shaya\Miniconda3\envs\graph_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
# Creating graphs from data

graph_karate = nx.karate_club_graph()

graph_dolphin = nx.Graph()

with open('soc-dolphins.txt', 'r') as file:
    for line in file:
        node1, node2 = line.strip().split()
        graph_dolphin.add_edge(node1, node2)

graph_football = nx.Graph()

with open('American_football.txt', 'r') as file:
    for line in file:
        node1, node2 = line.strip().split()
        graph_football.add_edge(node1, node2)


graph_book = nx.Graph()

with open('book.txt', 'r') as file:
    for line in file:
        node1, node2 = line.strip().split()
        graph_book.add_edge(node1, node2)



In [12]:
# Calculating adjancency and position
def get_adjacency_and_position(graph):
  adjacency = csr_matrix(nx.to_numpy_array(graph))
  position_dict = nx.spring_layout(graph)
  position = np.array(list(position_dict.values()))
  return adjacency, position


def get_node_array_without_embedding(graph):
  '''
  Convert graph to array without embedding
  '''
  node_array = to_numpy_array(graph)
  return node_array


# Node2vec
def get_embedded_node_array_using_node2vec(graph):

    model = Node2Vec(graph, dimensions=64, walk_length=80, num_walks=100, p=0.5, q=1.0)
    model = model.fit(window=10, min_count=1)

    embeddings = {node: model.wv[str(node)] for node in graph.nodes()}
    embedded_node_array = np.array(list(embeddings.values()))

    norm = np.linalg.norm(embedded_node_array, axis=1, keepdims=True)

    embedded_node_array = embedded_node_array / norm
    return embedded_node_array

# HOPE
def get_hope_embedding(graph, d=32):

    adjacency_matrix = nx.to_numpy_array(graph)
    identity_matrix = np.eye(adjacency_matrix.shape[0])

    katz_matrix = np.linalg.inv(identity_matrix - 0.5 * adjacency_matrix) - identity_matrix
    katz_matrix = np.asarray(katz_matrix)  

    svd = TruncatedSVD(n_components=d)
    U = svd.fit_transform(katz_matrix)
    S = svd.singular_values_
    V = svd.components_.T

    hope_embedding = np.concatenate((U * np.sqrt(S), V * np.sqrt(S)), axis=1)

    norm = np.linalg.norm(hope_embedding, axis=1, keepdims=True)
    
    hope_embedding = hope_embedding / norm
    return hope_embedding

# SDNE
def train_sdne(graph, hidden_layers=[256, 64], epochs=100, learning_rate=0.01):    
    class SDNE(nn.Module):
        def __init__(self, input_dim, hidden_layers, alpha=1e-5, beta=5):
            super(SDNE, self).__init__()
            self.alpha = alpha
            self.beta = beta
            self.encoder = nn.Sequential(
                nn.Linear(input_dim, hidden_layers[0]),
                nn.ReLU(),
                nn.Linear(hidden_layers[0], hidden_layers[1])
            )
            self.decoder = nn.Sequential(
                nn.Linear(hidden_layers[1], hidden_layers[0]),
                nn.ReLU(),
                nn.Linear(hidden_layers[0], input_dim)
            )

        def forward(self, x):
            encoded = self.encoder(x)
            reconstructed = self.decoder(encoded)
            return encoded, reconstructed

        def loss(self, x, reconstructed, L):
            bce_loss = nn.BCEWithLogitsLoss()(reconstructed, x)
            L_loss = self.beta * torch.mean(torch.sum(L * (x - reconstructed) ** 2, dim=1))
            return bce_loss + self.alpha * L_loss

    def get_adjacency_matrix(graph):
        adjacency_matrix = nx.to_numpy_array(graph)
        return adjacency_matrix

    adjacency_matrix = get_adjacency_matrix(graph)
    L = torch.FloatTensor(adjacency_matrix)
    D = np.diag(np.sum(adjacency_matrix, axis=1))
    L = D - adjacency_matrix
    L = torch.FloatTensor(L)

    model = SDNE(adjacency_matrix.shape[1], hidden_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    adjacency_matrix = torch.FloatTensor(adjacency_matrix)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        _, reconstructed = model(adjacency_matrix)
        loss = model.loss(adjacency_matrix, reconstructed, L)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        embedding, _ = model(adjacency_matrix)

    embedding = embedding.numpy()
    norm = np.linalg.norm(embedding, axis=1, keepdims=True)
    embedding = embedding / norm
    return embedding

In [38]:
# Creating data array

def prepare_data_arrays(graph):
    node2vec = get_embedded_node_array_using_node2vec(graph)
    hope = get_hope_embedding(graph)
    sdne = train_sdne(graph)
    
    return {
        'node_array': get_node_array_without_embedding(graph),
        'embedded_node_array_node2vec': node2vec,
        'embedded_node_array_hope': hope,
        'embedded_node_array_sdne': sdne,
        'embedded_node_array_sdne+hope': np.add(sdne, hope),
        'embedded_node_array_sdne+node2vec': np.add(sdne, node2vec),
        'embedded_node_array_hope+node2vec': np.add(hope, node2vec),
        'embedded_node_array_summation': np.add(np.add(sdne, hope), node2vec),
        'embedded_node_array_sdne*hope': np.multiply(sdne, hope),
        'embedded_node_array_sdne*node2vec': np.multiply(sdne, node2vec),
        'embedded_node_array_hope*node2vec': np.multiply(hope, node2vec),
        'embedded_node_array_multiply': np.multiply(np.multiply(sdne, hope), node2vec),
        'embedded_node_array_sdneMINhope': np.minimum(sdne, hope),
        'embedded_node_array_sdneMINnode2vec': np.minimum(sdne, node2vec),
        'embedded_node_array_hopeMINnode2vec': np.minimum(hope, node2vec),
        'embedded_node_array_minimum': np.minimum(np.minimum(sdne, hope), node2vec),
        'embedded_node_array_sdneMAXhope': np.maximum(sdne, hope),
        'embedded_node_array_sdneMAXnode2vec': np.maximum(sdne, node2vec),
        'embedded_node_array_hopeMAXnode2vec': np.maximum(hope, node2vec),
        'embedded_node_array_maximum': np.maximum(np.maximum(sdne, hope), node2vec),
    }

In [37]:
# Calculating adjancency and position for each dataset

graph = karate_club(metadata=True)
adjacency_karate, position_karate = graph.adjacency, graph.position
adjacency_dolphin, position_dolphin = get_adjacency_and_position(graph_dolphin)
adjacency_football, position_football = get_adjacency_and_position(graph_football)
adjacency_book, position_book = get_adjacency_and_position(graph_book)

In [35]:
# Creating data array for each dataset

karate_club_data_array = prepare_data_arrays(graph_karate)
dolphin_data_array = prepare_data_arrays(graph_dolphin)
football_data_array = prepare_data_arrays(graph_football)
book_data_array = prepare_data_arrays(graph_book)

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:03<00:00, 25.53it/s]


In [36]:
# K-means clustering

def perform_kmeans_clustering(data_array, adjacency, range_k=(2, 11), random_state=0):
    labels = []
    modularity = []
    work_log = []

    for k in range(*range_k):
        start_time = datetime.now()
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        kmeans.fit(data_array)
        end_time = datetime.now()

        labels.append(kmeans.labels_)
        modularity.append(get_modularity(adjacency, kmeans.labels_))
        work_log.append(end_time - start_time)

    best_k = modularity.index(max(modularity)) + range_k[0]
    best_labels = labels[best_k - range_k[0]]
    best_modularity = modularity[best_k - range_k[0]]
    best_work_log = work_log[best_k - range_k[0]]

    # print(f"Best value of k: {best_k}")
    return best_labels, best_modularity, best_work_log

def cluster_all_data_arrays(data_arrays, adjacency):
    labels_dict = {}
    modularity_dict = {}
    work_log_dict = {}

    for array_type, data_array in data_arrays.items():
        # print(f"Clustering for {array_type}...")
        best_labels, best_modularity, best_work_log = perform_kmeans_clustering(data_array, adjacency)
        
        labels_dict[array_type] = best_labels
        modularity_dict[array_type] = best_modularity
        work_log_dict[array_type] = best_work_log
    
    return labels_dict, modularity_dict, work_log_dict


In [44]:
# Performing K-means

kmeans_labels_dict_karate, kmeans_modularity_karate, kmeans_work_log_karate = cluster_all_data_arrays(karate_club_data_array, adjacency_karate)
kmeans_labels_dict_dolphin, kmeans_modularity_dolphin, kmeans_work_log_dolphin = cluster_all_data_arrays(dolphin_data_array, adjacency_dolphin)
kmeans_labels_dict_football, kmeans_modularity_football, kmeans_work_log_football = cluster_all_data_arrays(football_data_array, adjacency_football)
kmeans_labels_dict_book, kmeans_modularity_book, kmeans_work_log_book = cluster_all_data_arrays(book_data_array, adjacency_book)

In [29]:
kmeans_modularity_karate

{'node_array': 0.11119329388560151,
 'embedded_node_array_node2vec': 0.40327087442472065,
 'embedded_node_array_hope': 0.21572978303747536,
 'embedded_node_array_sdne': 0.13280736357659406,
 'embedded_node_array_sdne+hope': 0.22098948060486523,
 'embedded_node_array_sdne+node2vec': 0.40327087442472065,
 'embedded_node_array_hope+node2vec': 0.23528928336620644,
 'embedded_node_array_summation': 0.2693129520052597,
 'embedded_node_array_sdne*hope': 0.20603221564760033,
 'embedded_node_array_sdne*node2vec': 0.39907955292570674,
 'embedded_node_array_hope*node2vec': 0.1068376068376069,
 'embedded_node_array_multiply': 0.06730769230769218,
 'embedded_node_array_sdneMINhope': 0.19156804733727806,
 'embedded_node_array_sdneMINnode2vec': 0.3924227481919791,
 'embedded_node_array_hopeMINnode2vec': 0.21745562130177512,
 'embedded_node_array_minimum': 0.22238658777120318,
 'embedded_node_array_sdneMAXhope': 0.21194937541091385,
 'embedded_node_array_sdneMAXnode2vec': 0.39907955292570674,
 'embedd

In [30]:
kmeans_modularity_dolphin

{'node_array': 0.24597523832126889,
 'embedded_node_array_node2vec': 0.5231399074403703,
 'embedded_node_array_hope': 0.19057790435504934,
 'embedded_node_array_sdne': 0.10242870139630555,
 'embedded_node_array_sdne+hope': 0.1307701435860923,
 'embedded_node_array_sdne+node2vec': 0.3131996360903445,
 'embedded_node_array_hope+node2vec': 0.34021597246944346,
 'embedded_node_array_summation': 0.20155452711522492,
 'embedded_node_array_sdne*hope': 0.06578062576638585,
 'embedded_node_array_sdne*node2vec': 0.25380720699339426,
 'embedded_node_array_hope*node2vec': 0.12246351014595946,
 'embedded_node_array_multiply': 0.07102171591313644,
 'embedded_node_array_sdneMINhope': 0.13361813219413798,
 'embedded_node_array_sdneMINnode2vec': 0.28750840552193346,
 'embedded_node_array_hopeMINnode2vec': 0.32925912740793484,
 'embedded_node_array_minimum': 0.13747478343419958,
 'embedded_node_array_sdneMAXhope': 0.1307701435860923,
 'embedded_node_array_sdneMAXnode2vec': 0.2600965151694949,
 'embedded

In [31]:
kmeans_modularity_football

{'node_array': 0.5704156010740642,
 'embedded_node_array_node2vec': 0.5638530586610391,
 'embedded_node_array_hope': 0.058122144189649005,
 'embedded_node_array_sdne': 0.530166139303668,
 'embedded_node_array_sdne+hope': 0.22637577873640446,
 'embedded_node_array_sdne+node2vec': 0.5853343410446312,
 'embedded_node_array_hope+node2vec': 0.30940152061505877,
 'embedded_node_array_summation': 0.29857971253615917,
 'embedded_node_array_sdne*hope': 0.14256364947614086,
 'embedded_node_array_sdne*node2vec': 0.5575074048151923,
 'embedded_node_array_hope*node2vec': 0.062031460817683515,
 'embedded_node_array_multiply': 0.06946821052295454,
 'embedded_node_array_sdneMINhope': 0.22531262557581908,
 'embedded_node_array_sdneMINnode2vec': 0.5885530738299327,
 'embedded_node_array_hopeMINnode2vec': 0.26112052883553455,
 'embedded_node_array_minimum': 0.254424925951848,
 'embedded_node_array_sdneMAXhope': 0.20870135641843776,
 'embedded_node_array_sdneMAXnode2vec': 0.540415787358723,
 'embedded_nod

In [32]:
kmeans_modularity_book

{'node_array': 0.3640432741501741,
 'embedded_node_array_node2vec': 0.5119060473773789,
 'embedded_node_array_hope': 0.14496017605832962,
 'embedded_node_array_sdne': 0.4316617047423654,
 'embedded_node_array_sdne+hope': 0.27013949948838195,
 'embedded_node_array_sdne+node2vec': 0.4478046698649222,
 'embedded_node_array_hope+node2vec': 0.2721062725921813,
 'embedded_node_array_summation': 0.4404646212226385,
 'embedded_node_array_sdne*hope': 0.135481100981587,
 'embedded_node_array_sdne*node2vec': 0.4556563366087175,
 'embedded_node_array_hope*node2vec': 0.18724965420786588,
 'embedded_node_array_multiply': 0.09695034476375591,
 'embedded_node_array_sdneMINhope': 0.29376648618631107,
 'embedded_node_array_sdneMINnode2vec': 0.446817426895172,
 'embedded_node_array_hopeMINnode2vec': 0.33349530288305795,
 'embedded_node_array_minimum': 0.4247998519135544,
 'embedded_node_array_sdneMAXhope': 0.2874471028018162,
 'embedded_node_array_sdneMAXnode2vec': 0.4598932543538956,
 'embedded_node_arr